## POS tagging using modified Viterbi Algorithm:

In this project we apply techniques to improve Vanilla Viterbi Algorithm. Following are the steps followed:

1. plain vanilla Viterbi Algorithm is developed.
2. Viterbi Modification-Technique I - approach:
    * Transition probability is considered in case of unknown words.
    * The above approach is modified to consider transition probability weighted by tag occurrence probability in training set.
3. Viterbi Modification-Technique II - approach:
     * backoff to a rule based tagger in case of an unknown word.
     * The above technique is modified by using transition probability in approach 2 above if rule based tagger returns default noun tag ('NN').
4. The modified Viterbi algorithms are first tested on sampled test data for comparison.
5. The final algorithm and vanilla Viterbi Algorithm are the tested on full testing data for comparison.

### Data Preparation

In [1]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
# let's check some of the tagged data
print(nltk_data[1:5])

[[('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('conglomerate', 'NOUN'), ('.', '.')], [('A', 'DET'), ('form', 'NOUN'), ('of', 'ADP'), ('asbestos', 'NOUN'), ('once', 'ADV'), ('used', 'VERB'), ('*', 'X'), ('*', 'X'), ('to', 'PRT'), ('make', 'VERB'), ('Kent', 'NOUN'), ('cigarette', 'NOUN'), ('filters', 'NOUN'), ('has', 'VERB'), ('caused', 'VERB'), ('a', 'DET

In [4]:
# split data into training and validation set in the ratio 95:5

train_set,test_set = train_test_split(nltk_data,train_size=0.95,test_size=0.05,random_state = 101)

In [5]:
# create list of train and test tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
print(len(train_tagged_words))
print(len(test_tagged_words))

95547
5129


In [6]:
# check some of the tagged words.
train_tagged_words[1:5]

[('confirmed', 'VERB'), ('the', 'DET'), ('filing', 'NOUN'), ('but', 'CONJ')]

In [7]:
# let's check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)

12
{'X', 'DET', 'PRON', 'ADP', 'VERB', 'CONJ', '.', 'ADV', 'NOUN', 'NUM', 'ADJ', 'PRT'}


In [8]:
# let's check how many words are present in vocabulary
vocab = {word for word,tag in train_tagged_words}
print(len(vocab))

12100


### POS Tagging algorithm using Hidden Markov Model (HMM)

We'll use the HMM algorithm to tag the words. Given a sequence of words to be tagged, the task is to assign the most probable tag to the word. 
In other words, to every **word w**, assign **the tag t** that maximises the likelihood **P(t/w)**. 

Since P(t/w) = P(w/t). P(t) / P(w), after ignoring P(w), we have to compute P(w/t) and P(t).

Now:
* **P(w/t): is the emission probability** of a given word for a given tag. This can be computed based on the fraction of given word for given tag to the total count of that tag, ie: P(w/t) = count(w, t) / count(t). 

* **P(t): is the probability of tag (also transition probability)**, and in a tagging task, we assume that a tag will depend only on the previous tag (Markov order 1 assumption). In other words, the probability of say a tag being NN will depend only on the previous tag t(n-1). So for e.g. if t(n-1) is a JJ, then t(n) is likely to be an NN since adjectives often precede a noun (blue coat, tall building etc.).

### Build the Vanilla Viterbi based POS tagger

#### Function to compute emission probabilties for a given word

In [9]:
# compute emission probability for a given word for a given tag
def word_given_tag(word,tag,train_bag= train_tagged_words):
    taglist = [pair for pair in train_bag if pair[1] == tag] #to find all words having that tag
    tag_count = len(taglist)    
    #to find total occurences of a word belonging to any tag
    w_in_tag = [pair[0] for pair in taglist if pair[0]==word] 
    word_count_given_tag = len(w_in_tag)    
    
    return (word_count_given_tag,tag_count)

#### Function to compute transition probabilties for a given tag and previous tag

In [10]:
# compute transition probabilities of a previous and next tag
def t2_given_t1(t2,t1,train_bag=train_tagged_words):
    tags = [pair[1] for pair in train_bag] #finds all tags
    
    t1_tags = [tag for tag in tags if tag==t1]#finds a particular tag
    
    count_of_t1 = len(t1_tags)
    #finds a particular tag given the previous tag
    t2_given_t1 = [tags[index+1] for index in range(len(tags)-1) if tags[index] == t1 and tags[index+1] == t2]
    
    count_t2_given_t1 = len(t2_given_t1)
    
    return(count_t2_given_t1,count_of_t1)

In [11]:
t2_given_t1('NOUN','DET')

(5284, 8281)

In [12]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [13]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))

In [14]:
tags_df #table of transition probability

,X,DET,PRON,ADP,VERB,CONJ,.,ADV,NOUN,NUM,ADJ,PRT
X,0.076384,0.054742,0.055538,0.142584,0.203851,0.010662,0.163590,0.024984,0.062381,0.002864,0.017187,0.185232
DET,0.045405,0.005676,0.003744,0.009540,0.039850,0.000483,0.017993,0.012438,0.638087,0.022220,0.204323,0.000242
PRON,0.089969,0.009954,0.007657,0.022971,0.485452,0.005360,0.040965,0.034074,0.210949,0.006508,0.073124,0.013017
ADP,0.034427,0.324709,0.070031,0.016893,0.008340,0.000962,0.039025,0.014006,0.320967,0.062226,0.107024,0.001390
VERB,0.217506,0.134392,0.035786,0.092022,0.169249,0.005577,0.034934,0.081952,0.110070,0.022851,0.064988,0.030674
CONJ,0.008833,0.121339,0.058113,0.052534,0.156671,0.000465,0.034868,0.055323,0.349140,0.039981,0.118085,0.004649
.,0.026971,0.173335,0.066349,0.091342,0.089095,0.057538,0.093320,0.052324,0.222242,0.081003,0.043963,0.002427
ADV,0.023186,0.069891,0.014906,0.118582,0.343491,0.006956,0.137131,0.080490,0.031467,0.030474,0.129182,0.014243
NOUN,0.029175,0.012942,0.004607,0.176514,0.147667,0.042666,0.240604,0.017074,0.263564,0.009542,0.012248,0.043397
NUM,0.210542,0.003276,0.001489,0.036033,0.018761,0.013699,0.117332,0.002978,0.350208,0.184932,0.034247,0.026504


#### Viterbi Algorithm

The steps are as follows:

1. Given a sequence of words
2. iterate through the sequence
3. for each word (starting from first word in sequence) calculate the product of emission probabilties and transition probabilties for all possible tags.
4. assign the tag which has maximum probability obtained in step 3 above.
5. move to the next word in sequence to repeat steps 3 and 4 above.

In [15]:
# Vanilla Viterbi Algorithm
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                #since word before start of a sent is unknown, prev tag is '.'
                transition_p = tags_df.loc['.', tag] 
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

#### Testing Vanilla Viterbi Algorithm on sampled test data

In [16]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [17]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

accuracy = len(check)/len(tagged_seq)
print('Vanilla Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  16.17097306251526
Vanilla Viterbi Algorithm Accuracy:  89.38053097345133


In [18]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('Contra', 'X'), ('Contra', 'NOUN')),
 (('command', 'VERB'), ('command', 'NOUN')),
 (('Honduras', 'X'), ('Honduras', 'NOUN')),
 (('Sandinista', 'X'), ('Sandinista', 'NOUN')),
 (('offensive', 'X'), ('offensive', 'NOUN')),
 (('rebel', 'X'), ('rebel', 'NOUN')),
 (('forces', 'VERB'), ('forces', 'NOUN')),
 (('Bucking', 'X'), ('Bucking', 'VERB')),
 (('drew', 'X'), ('drew', 'VERB')),
 (('Eveready', 'X'), ('Eveready', 'NOUN')),
 (('complaining', 'X'), ('complaining', 'VERB')),
 (('up', 'ADV'), ('up', 'PRT'))]

### Solve the problem of unknown words
We can see that all of unknown words have been tagged as 'NUM' as 'NUM' is the first tag in tag list and is assigned
if unknown word is encountered (emission probability =0).

### Viterbi Modification-Technique I

* **First solution for unknown words**: assign based on transition probabilities only in case of unknown words as emission probability for unknown word is zero.

In [19]:
# use transition probability of tags when emission probability is zero (in case of unknown words)

def Viterbi_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # list for storing transition probabilities
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        
      
        # if probability is zero (unknown word) then use transition probability
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]
                           
        else:
            state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [20]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_1(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_1 Accuracy: ',accuracy*100)

Time taken in seconds:  16.315754175186157
Modified Viterbi_1 Accuracy:  94.69026548672566


In [21]:
# let's check the tags for words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')),
 (('Honduras', 'DET'), ('Honduras', 'NOUN')),
 (('Sandinista', 'VERB'), ('Sandinista', 'NOUN')),
 (('Eveready', 'VERB'), ('Eveready', 'NOUN')),
 (('*T*-252', 'VERB'), ('*T*-252', 'X')),
 (('up', 'ADV'), ('up', 'PRT'))]

**Adding Tag occurance probability weights**: 
we will apply weights based on the probability of tag occurance to the transition probabilities of tags and then use the resulting probability for predicting unknown words. 

This scheme will also take into account that some POS tags are more likely to occur as compared to others.

In [22]:
# lets create a list containing tuples of POS tags and POS tag occurance probability, based on training data
tag_prob = []
total_tag = len([tag for word,tag in train_tagged_words])
for t in tags:
    each_tag = [tag for word,tag in train_tagged_words if tag==t]
    tag_prob.append((t,len(each_tag)/total_tag))

tag_prob

[('X', 0.06576867928872701),
 ('DET', 0.08666938784053921),
 ('PRON', 0.0273373313657153),
 ('ADP', 0.0978889970381069),
 ('VERB', 0.1351167488251855),
 ('CONJ', 0.02251248076862696),
 ('.', 0.11641391147812072),
 ('ADV', 0.031597015081582885),
 ('NOUN', 0.2862674913916711),
 ('NUM', 0.035145007169246546),
 ('ADJ', 0.06351847781719991),
 ('PRT', 0.03176447193527793)]

In [23]:
def Viterbi_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # list for storing transition probabilities
       
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
            # find POS tag occurance probability
            tag_p = [pair[1] for pair in tag_prob if pair[0]==tag ]
            
            # calculate the transition prob weighted by tag occurance probability.
            transition_p = tag_p[0]*transition_p             
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        
      
        # if probability is zero (unknown word) then use weighted transition probability
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]                 
                           
        else:
            state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [24]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_1(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_1 Accuracy: ',accuracy*100)

Time taken in seconds:  17.234552145004272
Modified Viterbi_1 Accuracy:  95.57522123893806


**Thus, we see that we have got a much better accuracy by using weighted transition probabilties.**

In [25]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')),
 (('Sandinista', 'VERB'), ('Sandinista', 'NOUN')),
 (('Eveready', 'VERB'), ('Eveready', 'NOUN')),
 (('*T*-252', 'VERB'), ('*T*-252', 'X')),
 (('up', 'ADV'), ('up', 'PRT'))]

The following list of words have been **correctly POS tagged by Viterbi_1** as compared to vanilla Viterbi Algorithm:

* Contra:correctly tagged as NOUN
* Honduras:correctly tagged as NOUN
* complaining: correctly tagged as VERB
* Bucking: correctly tagged as VERB

### Viterbi Modification-Technique II

**second solution for unknown words:** 

* backoff to rule based tagger in case of unknown words.
* we further observe that POS tag 'X' can be easily encapsulated in regex rule, so we extract it only based on ruled based tagged.

Let's define a rule based tagger as below:

In [26]:
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
]

# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [27]:
# Modification in Viterbi Algorithm : 
#Backoff to rule based tagger in case unknown word is encountered.
def Viterbi_2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
       
        
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]                
            
        
        state.append(state_max)
    return list(zip(words, state))

In [28]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  14.683353185653687


In [29]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_2 Accuracy: ',accuracy*100)

Modified Viterbi_2 Accuracy:  97.34513274336283


In [30]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')),
 (('drew', 'NOUN'), ('drew', 'VERB')),
 (('up', 'ADV'), ('up', 'PRT'))]

The following list of words have been correctly POS tagged by Viterbi_2 as compared to vanilla Viterbi Algorithm:

* Contra:correctly tagged as NOUN
* Honduras:correctly tagged as NOUN
* complaining: correctly tagged as VERB
* Bucking: correctly tagged as VERB

the following list of words has been correctly tagged by Viterbi_2 as compared to Viterbi_1

* Sandinista: correctly tagged as NOUN
* Eveready: correctly tagged as NOUN
* `*T*-252`: correctly tagged as 'X'

**further modification in Viterb_2:** We know that the rule based tagger assigns 'NOUN' by default if word does not fall in any rule, to correct this let's assign the tags for any such word based purely on transition probability of tags.

So, first we will modify the rule based tagger to output 'NN' instead of 'NOUN' in case word does not satisfy any rules. We also observe that any capitalized word can still be defaulted as 'NOUN' so will add one more rule for that case.

In [31]:
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'^[A-Z][a-z].*', 'NOUN'),       # NOUN
    (r'.*', 'NN')                     # default
]

# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [32]:
# modified Viterbi
def Viterbi_2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # for storing transition probabilities
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
            # find POS tag occurance probability
            tag_p = [pair[1] for pair in tag_prob if pair[0]==tag ]
            
            # calculate the transition prob weighted by tag occurance probability.
            transition_p = tag_p[0]*transition_p
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1] 
        
      
        # getting state for which probability is maximum
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
            
            # if unknown word does not satisfy any rule, find the tag with maximum transition probability
            if state_max == 'NN':
                pmax = max(p_transition)
                state_max = T[p_transition.index(pmax)]                 
                
        else:
             if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [33]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  14.343031883239746


In [34]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

Modified Viterbi Algorithm Accuracy:  98.23008849557522


We observe that much better accuracy is obtained now.

In [35]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')), (('up', 'ADV'), ('up', 'PRT'))]

The following list of words have been correctly POS tagged by Viterbi_2 as compared to vanilla Viterbi Algorithm:
* Contra:correctly tagged as NOUN
* Honduras:correctly tagged as NOUN
* complaining: correctly tagged as VERB
* Bucking: correctly tagged as VERB

the following list of words has been correctly tagged by Viterbi_2 as compared to Viterbi_1
* Sandinista: correctly tagged as NOUN
* Eveready: correctly tagged as NOUN
* `*T*-252`: correctly tagged as 'X'
* drew: correctly tagged as VERB

#### Evaluate vanilla Viterbi and modified Viterbi on entire test data

In [28]:
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
test_run_base = [tup for sent in test_set for tup in sent]

In [ ]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

In [ ]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

#### Evaluating tagging on sample 'Test_sentences.txt' file

In [39]:
f = open('C:/Users/ArrunPersonal/Codes/Semester5/Sem5_MISProject/SampleTextDoc.txt')

In [40]:
text = f.read()

In [41]:
sample_test_sent = text.splitlines()

In [42]:
f.close()

In [43]:
sample_test_sent[:-3]

["The UEFA Euro 2008 Final was the final match of Euro 2008, the thirteenth edition of the European Football Championship, UEFA's competition for national football teams. The match was played at Ernst-Happel-Stadion, Vienna, Austria, on 29 June 2008, and was contested by Germany and Spain. The sixteen-team tournament consisted of a group stage, from which eight teams qualified for the knockout phase. En route to the final, Germany finished second in Group B, with a defeat to Croatia and wins over Poland and Austria, after which they defeated Portugal and Turkey in the knockouts. Spain finished top of Group D with three wins, against Russia, Sweden and Greece, before defeating Italy on penalties in the quarter-final and a second victory over Russia in the semi-final.",
 '']

In [44]:
# list of untagged words
sample_test_words = [word for sent in sample_test_sent for word in sent.split()]

In [45]:
# tagging the test sentences
start = time.time()
sample_tagged_seq = Viterbi(sample_test_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  43.91729426383972


In [46]:
sample_tagged_seq[:25]

[('The', 'DET'),
 ('UEFA', 'NOUN'),
 ('Euro', 'NOUN'),
 ('2008', 'NOUN'),
 ('Final', 'NOUN'),
 ('was', 'VERB'),
 ('the', 'DET'),
 ('final', 'ADJ'),
 ('match', 'NOUN'),
 ('of', 'ADP'),
 ('Euro', 'NOUN'),
 ('2008,', 'NOUN'),
 ('the', 'DET'),
 ('thirteenth', 'NOUN'),
 ('edition', 'NOUN'),
 ('of', 'ADP'),
 ('the', 'DET'),
 ('European', 'ADJ'),
 ('Football', 'NOUN'),
 ('Championship,', 'NOUN'),
 ("UEFA's", 'NOUN'),
 ('competition', 'NOUN'),
 ('for', 'ADP'),
 ('national', 'ADJ'),
 ('football', 'NOUN')]

We can see that several words have been misclassified by vanilla Viterbi POS tagger, for example:
* Android as NUM
* Google as NUM
* OS as NUM

In [47]:
# tagging the test sentences
start = time.time()
sample_tagged_seq = Viterbi_2(sample_test_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  47.281858921051025


In [48]:
sample_tagged_seq[:25]

[('The', 'DET'),
 ('UEFA', 'NOUN'),
 ('Euro', 'NOUN'),
 ('2008', 'NUM'),
 ('Final', 'NOUN'),
 ('was', 'VERB'),
 ('the', 'DET'),
 ('final', 'ADJ'),
 ('match', 'NOUN'),
 ('of', 'ADP'),
 ('Euro', 'NOUN'),
 ('2008,', 'NOUN'),
 ('the', 'DET'),
 ('thirteenth', 'NOUN'),
 ('edition', 'NOUN'),
 ('of', 'ADP'),
 ('the', 'DET'),
 ('European', 'ADJ'),
 ('Football', 'NOUN'),
 ('Championship,', 'NOUN'),
 ("UEFA's", 'NOUN'),
 ('competition', 'NOUN'),
 ('for', 'ADP'),
 ('national', 'ADJ'),
 ('football', 'NOUN')]

All these cases were correctly POS tagged by Viterbi_2:
* Android as NOUN
* Google as NOUN
* OS as NOUN

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

The accuracy of vanilla Viterbi Algorithm: **91.52%**

The accuracy of modified Viterbi Algorithm: **95.44%**

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

The following cases were incorrectly tagged which got corrected by modified Viterbi Algorithm:

* Contra:correctly tagged as NOUN
* Honduras:correctly tagged as NOUN
* complaining: correctly tagged as VERB
* Bucking: correctly tagged as VERB
* Sandinista: correctly tagged as NOUN
* Eveready: correctly tagged as NOUN
* `*T*-252`: correctly tagged as 'X'
* drew: correctly tagged as VERB